# Sberbank version of AutoML (AI Lab production) 

Данное решение представляет собой Python библиотеку, позволяющую автоматически обучать модель и проверять их качество на тестовых датасетах.

**!!!! Данная версия не имеет обратной совместимости с предыдущей, комментарии см. ниже**

**V1.3.0. Обновления по сравнению с предыдущей версией:**

1. Отсутсвует аргумент model_dir при создании AutoML. При тренировке и инференсе на диск ничего не сохраняется. Исключение - если указана cahce_dir
2. Появилась возможность указаня cache_dir для train/predict. Аргумент не обязателен, но повышает эффективность обработки текстов. Эту директорию можно удалить по завершении работы, а можно оставить для ускорения повторных экспериментов на тех же данных. 
3. Появилась возможность обучать модели из памяти (передавать DataFrame) без сохранения его в csv. Интерфейс точно такой же, только вместо пути до файла можно передать другой объект
4. Появилась возможность ускорять инференс за счет параллельного выполнения. Аргумент - n_jobs
5. Появился sklearn подобный интерфейс (см. Package_usage_sklearn)
6. Для сохранения и загрузки моделей методам .save и .load теперь надо передавать путь до файла модели. Так же теперь возможно сохранение при помощи Pickle 

Еще раз отмечу здесь **важное замечание**, которое необходимо учитывать при работе с библиотекой: в случае, если используемый для тренировки датасет не помешается в заданную память, в логе будет выведено сообщение "We are sampling the data to fit the memory" - это означает, что датасет для непосредственной тренировки моделей был семплирован из исходного для того, чтобы уместиться в заданную память. 

При решении задачи регрессии можно полностью игнорировать данный факт (либо поставить больше памяти чтобы использовать больше данных для обучения).

При решении задачи классификации данный факт можно игнорировать в случае, если необходимо просто оценить метрику полученной модели (Gini, AUC и т.п.). Если планируется использовать полученные из модели вероятности в дальнейшем бизнес процессе, то **необходимо самостоятельно сделать для них калибровку**.

In [1]:
import time
START_TIME = time.time()

# Package installation 

Библиотеку можно установить для текущего юзера следующей командой:

In [2]:
!pip uninstall --yes sber_ailab_automl 
!python setup.py install --user

Cannot uninstall requirement sber-ailab-automl, not installed
pip install -i http://mirror.ca.sbrf.ru/pypi/simple/ --trusted-host mirror.ca.sbrf.ru --user -U numpy scipy==1.2.0 pandas scikit-learn lightgbm hyperopt psutil shap==0.27.0 setuptools==40.5.0 xgboost joblib torch==1.0.0 gensim==3.6.0 pymystem3 tqdm seaborn jinja2
Requirement already up-to-date: numpy in /home/ALPHA/penkin-go/.local/lib/python3.6/site-packages
Requirement already up-to-date: scipy==1.2.0 in /home/ALPHA/penkin-go/.local/lib/python3.6/site-packages
Requirement already up-to-date: pandas in /home/ALPHA/penkin-go/.local/lib/python3.6/site-packages
Requirement already up-to-date: scikit-learn in /home/ALPHA/penkin-go/.local/lib/python3.6/site-packages
Requirement already up-to-date: lightgbm in /home/ALPHA/penkin-go/.local/lib/python3.6/site-packages
Requirement already up-to-date: hyperopt in /home/ALPHA/penkin-go/.local/lib/python3.6/site-packages
Requirement already up-to-date: psutil in /home/ALPHA/penkin-go/.

Adding sber-ailab-automl 1.3.4 to easy-install.pth file
Installing saa-predict script to /home/ALPHA/penkin-go/.local/bin
Installing saa-train script to /home/ALPHA/penkin-go/.local/bin

Installed /home/ALPHA/penkin-go/.local/lib/python3.6/site-packages/sber_ailab_automl-1.3.4-py3.6.egg
Processing dependencies for sber-ailab-automl==1.3.4
Finished processing dependencies for sber-ailab-automl==1.3.4


# Necessary imports 

In [3]:
import sber_ailab_automl as saa
import numpy as np
import pandas as pd
import logging
import sys

from sklearn.metrics import roc_auc_score, mean_squared_error

logging.basicConfig(level=logging.INFO, stream=sys.stdout,
                        format='[%(asctime)-15s] %(levelname)-8s: %(message)s')
logger = logging.getLogger(__name__)

/home/ALPHA/penkin-go/.local/lib/python3.6/site-packages/sklearn/externals/joblib/__init__.py:15: DeprecationWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=DeprecationWarning)


In [4]:
saa.__version__

'1.3.4'

# Set parameters 

Опишем те параметры, которые можно передать решению для работы:

- `params['vCPULimit']` - ограничение по количеству используемых виртуальных ядер процессора
- `params['memoryLimit']` - ограничение на используемую память
- `params['timeLimit']` - ограничение на время работы
- `params['type']` - тип задачи ('classification' или 'regression')
- `params['trainDatasetFName']` - путь до файла с тренировочной выборкой 
- `params['testDatasetFName']` - путь до файла с тестовой выборкой
- `params['predictionsFName']` - путь до файла с итоговыми предсказаниями
- `params['encoding']` - кодировка, в которой сохранены тренировочный и тестовый файлы (для Unix систем это обычно `'utf-8'`, а для Windows - `'cp1251'`)
- `params['separator']` - разделитель полей в CSV файле (обычно `r','`, а для файла, экспортированного из Teradata - `r'\t'`). Букву r перед параметром необходимо сохранить для правильного чтения параметра (в сам параметр он не входит)
- `params['decimal']` - разделитель целой и дробной части для вещественных чисел (обычно `'.'`, а для файла, экспортированного из Teradata - `','`)
- `params['debugFlag']` - флаг debug режима (если True, то печатается лог тренировки; если False - то нет)
- `params['featureRoles']` - параметр, в котором указываются роли и типы переменных в виде словаря:
    * `target` - строка с названием колонки целевой переменной (**обязательный параметр**)
    * `line_id` - строка с названием колонки id строки (необязательный параметр)
    * `group` - строка с названием колонки-группы для объекта, используется для GroupKFold валидации (необязательный параметр). **Важное замечание**: пока что поддерживается только одна колонка в качестве группы, а также данная колонка НЕ должна содержать пропущенных значений
    * `drop` - массив названий колонок, которые необходимо удалить из датасета (необязательный параметр)
    * `numeric` - массив названий колонок, которые являются вещественными признаками (необязательный параметр)
    * `datetime` - массив названий колонок, которые являются датами (в указанном в параметре `params['datetimeFormat']` формате) (необязательный параметр)
    * `string` - массив названий колонок, которые являются строками (категориальными признаками) (необязательный параметр)
    * `id` - массив названий колонок, которые являются id-шниками (необязательный параметр)
    * `text` - массив названий колонок, которые являются текстами (необязательный параметр)
- `params['naValues']` - символ пропущенного значения (`None` если пропуски в CSV файле являются пустыми)
- `params['rowsToAnalyze']` - количество строчек, по которым автоматически определяются типы данных
- `params['datetimeFormat']` - формат, в котором в файле записаны даты (более подробно о форматах можно посмотреть [здесь](https://docs.python.org/3/library/datetime.html#strftime-strptime-behavior))
- `params['useGPU']` - флаг использования GPU в задаче с текстовыми данными (будет работать только при подходящей под установленную CUDA версии torch)
- `params['testBatchSize']` - размер батча, используемый для предсказания тестовой выборки
- `params['LuckySeed']` - фиксируемый seed для построения воспроизводимой кросс-валидации (также может быть использован при построении ансамбля AutoML моделей, построенных на разных CV разбиениях)
- `params['KFolds']` - количество частей, на которые разбивается выборка в процессе решения задачи

In [5]:
params = {}
params['vCPULimit'] = 20
params['memoryLimit'] = 32 * 1024
params['timeLimit'] = 3000 * 60
params['type'] = 'classification'
params['trainDatasetFName'] = 'example_data/Train.csv'
params['testDatasetFName'] = 'example_data/Test.csv'
params['predictionsFName'] = 'preds.csv'
params['encoding'] = 'utf-8'
params['separator'] = r","
params['decimal'] = '.'
params['debugFlag'] = True
params['featureRoles'] = {
    'target': 'target',
    #'group': 'tab_num',
    'drop': [],
    'numeric': [],
    'datetime': [],
    'string': [],
    'id': [],
    'text': []
}
params['naValues'] = None
params['rowsToAnalyze'] = 1000000
params['datetimeFormat'] = '%Y-%m-%d'
params['testBatchSize'] = 10000
params['LuckySeed'] = 777
params['KFolds'] = 5

# Validation report generation

Для автоматической генерации отчета необходимо заполнить поля в ячейке ниже - они будут использованы в отчете в том виде, в котором указаны. Заполнение всех значений является обязательным.

Параметр `automl_date_column` имеет под собой следующую логику - по нему будут проставлены временные периоды, за которые взяты все используемые выборки (как тренировочная, так и ООТ тестовая).

**Важное замечание: на текущий момент построитель отчетов работает только с задачами бинарной классификации, в которых в качестве тестовой используется Out-of-Time выборка. Расширение возможностей автоматического построителя отчетов находится в разработке - stay tuned :)**

In [ ]:
report_params = {}
# PARAMS FOR AUTOML MODEL
report_params['automl_params'] = params # параметры AutoML
report_params['automl_date_column'] = 'report_month' # колонка с датой в формате params['datetimeFormat']

# REPORT PARAMS
report_params['output_path'] = "ValidReport" # папка, куда сгенерится отчет и сложатся нужные файлы
report_params['report_name'] = "___НАЗВАНИЕ ОТЧЕТА___"
report_params['report_version_id'] = 1
report_params['city'] = 'Москва'
report_params['model_aim'] = '___ЦЕЛЬ ПОСТРОЕНИЯ МОДЕЛИ___'
report_params['model_name'] = '___НАЗВАНИЕ МОДЕЛИ___'
report_params['zakazchik'] = '___ЗАКАЗЧИК___'
report_params['high_level_department'] = '___ПОДРАЗДЕЛЕНИЕ___'
report_params['ds_name'] = '___РАЗРАБОТЧИК МОДЕЛИ___'
report_params['target_descr'] = '___ОПИСАНИЕ ЦЕЛЕВОГО СОБЫТИЯ___'
report_params['non_target_descr'] = '___ОПИСАНИЕ НЕЦЕЛЕВОГО СОБЫТИЯ___'

RG = saa.ReportGenerator()
RG.generate_report(report_params)

In [11]:
USED_TIME = time.time() - START_TIME
print('THIS NOTEBOOK TOOK {:.3f} hours'.format(USED_TIME / 3600))

THIS NOTEBOOK TOOK 0.009 hours


# Открыть отчет в браузере 

In [ ]:
import webbrowser
import os
result_file = os.path.join(report_params['output_path'], 'report.html')
webbrowser.open(result_file)

# Что лежит в папке с отчетом:
1. Сам отчет report.html
2. Библиотека `shap.js` для отображения примеров интерпретации объектов тестовой выборки
3. Необходимые графики
4. Тренировочная и валидационная выборки, сгенерированные из тренировочной выборки, для проведения честной OOS валидации (в отношении 80:20)
5. Логи построения моделей (см. Приложение 1 в конце валидационного отчета)
6. Обученные модели - как на полной выборке, так и на 80% тренировочной выборки(см. пункт 4)